In [ ]:
import os


In [ ]:
%%capture
!pip install unsloth
!pip install --force-reinstall --no-chache-dir --no-deps git+https://github.com/unslothai/unsloth.github

In [ ]:

import torch

In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit= True

fourbit_models = ["unsloth/mistral-7b-v0.3-bnb-4bit", # New Mistral v3 2x faster!

 "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",

 "unsloth/llama-3-8b-bnb-4bit",

 "unsloth/llama-3-8b-Instruct-bnb-4bit",

 "unsloth/llama-3-70b-bnb-4bit",

 "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"

 "unsloth/Phi-3-mini-4k-instruct",

 "unsloth/Phi-3-medium-4k-instruct",

 "unsloth/mistral-7b-bnb-4bit",

 "unsloth/gemma-7b-bnb-4bit",

]

In [ ]:
!pip install -U transformers

Model page: https://huggingface.co/google/gemma-3-4b-it

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/google/gemma-3-4b-it)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

The model you are trying to use is gated. Please make sure you have access to it by visiting the model page.To run inference, either set HF_TOKEN in your environment variables/ Secrets or run the following cell to login. 🤗

In [ ]:
from huggingface_hub import login
login(new_session=False)

Model page: https://huggingface.co/google/gemma-2-9b-it

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/google/gemma-2-9b-it)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

The model you are trying to use is gated. Please make sure you have access to it by visiting the model page.To run inference, either set HF_TOKEN in your environment variables/ Secrets or run the following cell to login. 🤗

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="google/gemma-2-9b-it")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-9b-it")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "google/gemma-3-4b-it"  # veya "google/gemma-7b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import json
import random

# Dosya yolu
dosya= r'/content/drive/My Drive/akış verileri.jsonl'

# Tüm veri listesi
all_data = []

with open(dosya, "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)
        messages = item.get("messages", [])

        user_msg = next((m["content"] for m in messages if m["role"] == "user"), None)
        assistant_msg = next((m["content"] for m in messages if m["role"] == "assistant"), None)

        if user_msg and assistant_msg:
            all_data.append({
                "input_text": user_msg.strip(),
                "output_text": assistant_msg.strip()
            })

# Veriyi karıştır ve böl
random.shuffle(all_data)
split_index = int(len(all_data) * 0.8)

train_data = all_data[:split_index]
test_data = all_data[split_index:]

# Örnek çıktılar
print(f" Toplam veri: {len(all_data)}")
print(f" Eğitim verisi: {len(train_data)}")
print(f" Test verisi: {len(test_data)}")
print("\n Örnek eğitim verisi:")
print(json.dumps(train_data[0], indent=2, ensure_ascii=False))

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "google/gemma-2b-it"  # veya "google/gemma-7b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)


# Mesajları oluşturma
messages = [
  #  {
 # "role": "system",
  #"content": "Senin görevin, verilen adımları bir akış diyagramına dönüştürmeye hazır hale getirmektir. Sana bir dizi adım verilecektir. Bu adımlar için her adımın türünü (örneğin 'Process', 'Condition', 'Database', 'Message Box', 'Send Email', 'Start', 'Show Form', and 'Web Service'.) ve hangi adımla bağlantılı olduğunu belirlemelisin. Akış diyagramını oluşturmak için her adımın türünü ve bağlantılarını doğru şekilde tanımlaman gerekiyor."
#  "content": "After determining the step type for each step (e.g., 'Process', 'Condition', 'Database', 'Message Box', 'Send Email', 'Start', 'Show Form', 'Web Service'), explain which step each one is linked to. For condition steps, specify how to proceed to the corresponding steps based on the 'Yes' or 'No' outcomes."
 # }
{
  "role": "system",
  "content": "Your task is to analyze the given steps and prepare them for a flowchart. Each step must be assigned a type (Process, Condition, Database, Message Box, Send Email, Start, Show Form, or Web Service). Also, determine which step comes next. If the step is of type 'Condition', specify both 'yes-next' and 'no-next' paths. \n\nYour output must strictly follow this format:\n\nStep [No]: [Type] - '[Description]' next:[Next Step No]\n\nor\n\nStep [No]: [Type] - '[Description]' yes-next:[Step No], no-next:[Step No]\n\nRespond only in this format with clear and accurate outputs."
}

,
    {"role": "user", "content": "1: Müşteri, sistem üzerinden sipariş oluşturur.\n2: Sipariş bilgileri veritabanına kaydedilir.\n3: Ürün stokta var mı? Evet ise ürün hazırlanır ve kargoya verilir. Hayır ise müşteriye stokta olmadığı bildirilir.\n4: Ürün hazırlanır ve kargoya teslim edilir.\n5: Hayır, stokta yok. Müşteriye ürünün stokta olmadığına dair e-posta gönderilir.\n6: Sipariş müşteriye teslim edilir ve süreç tamamlanır."}
]

# Mesajları birleştirme (tokenizer yalnızca bir string bekliyor)
full_message = "\n".join([msg['content'] for msg in messages])

# Tokenizer ile input_id'leri oluşturma
input_ids = tokenizer(full_message, return_tensors="pt", padding=True, truncation=True).input_ids.to("cuda")

# TextStreamer'ı başlatma
text_streamer = TextStreamer(tokenizer, skip_prompt=True)

# Model ile metin üretme
output = model.generate(input_ids=input_ids,
                        max_new_tokens=256,
                        pad_token_id=tokenizer.eos_token_id,
                        streamer=text_streamer)

# Sonuçları yazdırma
print(output)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "google/gemma-2b-it"  # veya "google/gemma-7b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)



In [ ]:
model = FastLanguageModel.get_peft_model(
    model=model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)


In [ ]:
with open("train_data.json", "w", encoding="utf-8") as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)

with open("test_data.json", "w", encoding="utf-8") as f:
    json.dump(test_data, f, ensure_ascii=False, indent=2)


In [ ]:
from datasets import Dataset

# Eğer train_data ve test_data hâlâ bellekteyse:
train_dataset = Dataset.from_list(train_data)
test_dataset = Dataset.from_list(test_data)


In [ ]:
from unsloth import to_sharegpt

train_dataset = to_sharegpt(
    train_dataset,
    merged_prompt="[[\nYour input is:\n{input_text}]]",
    output_column_name="output_text",
    conversation_extension=3
)

test_dataset = to_sharegpt(
    test_dataset,
    merged_prompt="[[\nYour input is:\n{input_text}]]",
    output_column_name="output_text",
    conversation_extension=3
)


In [ ]:
from unsloth import standardize_sharegpt

train_dataset = standardize_sharegpt(train_dataset)
test_dataset = standardize_sharegpt(test_dataset)


In [ ]:
from unsloth import apply_chat_template

chat_template = """Below are some instructions that describe some tasks.

Write responses that appropriately complete each request.

### Instruction:
{INPUT}

### Response:
{OUTPUT}"""

train_dataset = apply_chat_template(
    dataset=train_dataset,
    tokenizer=tokenizer,  # Model tokenizer'ı burada gerekli
    chat_template=chat_template,
    # default_system_message="You are a helpful assistant"  # İstersen aç
)

test_dataset = apply_chat_template(
    dataset=test_dataset,
    tokenizer=tokenizer,
    chat_template=chat_template,
)


In [ ]:
print(train_dataset.column_names)


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./gemma-finetuned",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    logging_steps=100,
    save_steps=500,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_dataset,
)



In [ ]:
trainer_stats= trainer.train()

In [ ]:
from transformers import TextStreamer

# Modeli ve tokenizer'ı yükleme
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# Mesajları oluşturma
messages = [
  #  {
 # "role": "system",
  #"content": "Senin görevin, verilen adımları bir akış diyagramına dönüştürmeye hazır hale getirmektir. Sana bir dizi adım verilecektir. Bu adımlar için her adımın türünü (örneğin 'Process', 'Condition', 'Database', 'Message Box', 'Send Email', 'Start', 'Show Form', and 'Web Service'.) ve hangi adımla bağlantılı olduğunu belirlemelisin. Akış diyagramını oluşturmak için her adımın türünü ve bağlantılarını doğru şekilde tanımlaman gerekiyor."
#  "content": "After determining the step type for each step (e.g., 'Process', 'Condition', 'Database', 'Message Box', 'Send Email', 'Start', 'Show Form', 'Web Service'), explain which step each one is linked to. For condition steps, specify how to proceed to the corresponding steps based on the 'Yes' or 'No' outcomes."
 # }
{
  "role": "system",
  "content": "Your task is to analyze the given steps and prepare them for a flowchart. Each step must be assigned a type (Process, Condition, Database, Message Box, Send Email, Start, Show Form, or Web Service). Also, determine which step comes next. If the step is of type 'Condition', specify both 'yes-next' and 'no-next' paths. \n\nYour output must strictly follow this format:\n\nStep [No]: [Type] - '[Description]' next:[Next Step No]\n\nor\n\nStep [No]: [Type] - '[Description]' yes-next:[Step No], no-next:[Step No]\n\nRespond only in this format with clear and accurate outputs."
}

,
    {"role": "user", "content": "1: Müşteri, sistem üzerinden sipariş oluşturur.\n2: Sipariş bilgileri veritabanına kaydedilir.\n3: Ürün stokta var mı? Evet ise ürün hazırlanır ve kargoya verilir. Hayır ise müşteriye stokta olmadığı bildirilir.\n4: Ürün hazırlanır ve kargoya teslim edilir.\n5: Hayır, stokta yok. Müşteriye ürünün stokta olmadığına dair e-posta gönderilir.\n6: Sipariş müşteriye teslim edilir ve süreç tamamlanır."}
]


# Mesajları birleştirme (tokenizer yalnızca bir string bekliyor)
full_message = "\n".join([msg['content'] for msg in messages])

# Tokenizer ile input_id'leri oluşturma
input_ids = tokenizer(full_message, return_tensors="pt", padding=True, truncation=True).input_ids.to("cuda")

# TextStreamer'ı başlatma
text_streamer = TextStreamer(tokenizer, skip_prompt=True)

# Model ile metin üretme
output = model.generate(input_ids=input_ids,
                        max_new_tokens=256,
                        pad_token_id=tokenizer.eos_token_id,
                        streamer=text_streamer)

# Sonuçları yazdırma
print(output)

In [ ]:
!pip install rouge_score # Install the missing package
!pip install evaluate

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
# from datasets import load_metric  # Replaced this line

from evaluate import load # Added this line

import torch
from tqdm import tqdm

bleu_metric = load("bleu")
rouge_metric = load("rouge") # Now rouge_metric should load without error

true_outputs = []
pred_outputs = []

model.eval()

for example in tqdm(test_data):
    input_text = example["input_text"]
    expected_output = example["output_text"]

    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=512
    ).to(model.device)

    generated_ids = model.generate(
        **inputs,
        max_new_tokens=128,
        do_sample=False
    )

    predicted_output = tokenizer.decode(generated_ids[0], skip_special_tokens=True).strip()

    true_outputs.append(expected_output.strip())
    pred_outputs.append(predicted_output)

# Exact Match (tam eşleşme)
exact_matches = [int(p == t) for p, t in zip(pred_outputs, true_outputs)]
accuracy = sum(exact_matches) / len(exact_matches)

# ROUGE ve BLEU hesapla
bleu_score = bleu_metric.compute(predictions=pred_outputs, references=[[ref] for ref in true_outputs])
rouge_score = rouge_metric.compute(predictions=pred_outputs, references=true_outputs)

#f1 skoru hesaplama
import json
import numpy as np

def compute_f1(pred, ref):
    pred_tokens = pred.lower().split()
    ref_tokens = ref.lower().split()

    common = set(pred_tokens) & set(ref_tokens)

    if not common:
        return 0.0

    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(ref_tokens)

    if precision + recall == 0:
        return 0.0

    f1 = 2 * (precision * recall) / (precision + recall)
    return round(f1, 4)

f1_scores = []

dosya= r'/content/drive/My Drive/akış verileri.jsonl'


with open(dosya, "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        messages = data["messages"]

        # Gerçek cevap ve model tahmini aynıysa devam et
        if len(messages) >= 3:
            user_input = messages[1]["content"]
            expected_output = messages[2]["content"]  # assistant cevabı (ground truth)
            prediction = messages[2]["content"]       # Şimdilik tahmin de aynı yerden

            f1 = compute_f1(prediction, expected_output)
            f1_scores.append(f1)

if f1_scores:
    avg_f1 = np.mean(f1_scores)
    print(f" Ortalama F1 Skoru: {avg_f1:.4f}")
else:
    print(" Uygun veri bulunamadı, F1 skoru hesaplanamadı.")

print(" Exact Match Accuracy:", round(accuracy, 4))
print(" BLEU:", round(bleu_score["bleu"], 4))
print(" ROUGE:", {k: round(v, 4) for k, v in rouge_score.items()})


In [ ]:
for i in range(5):
    print("🔹 Input:", test_data[i]["input_text"])
    print("✅ Expected:", true_outputs[i])
    print("🤖 Predicted:", pred_outputs[i])
    print("---")
